In [1]:
#Import Required Libraries
import keras # ML Model library
import numpy as np #linear algebra
import pandas as pd #Data processing, CSV file I/O (e.g. pd.read_csv)
import xarray as xr #Data Storage as N-D Dataset
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau

#Pre-run configuration
np.random.seed(56)

#Model Configuration
image_width, image_height = 28, 28
batch_size = 300
no_epochs = 30
no_classes = 10
validation_split = 0.2
verbosity = 1
base_filters = 86


Using TensorFlow backend.


In [2]:
#Import Training and Test datasets
train = pd.read_csv("../input/digit-recognizer/train.csv")
finaltest = pd.read_csv("../input/digit-recognizer/test.csv")
train['label'].value_counts(normalize=False)

1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: label, dtype: int64

In [3]:
#Prepare Datalists
traindatalist = []
y_trainlist = []
#y_trainstack =
y_trainlabels = []
x_trainlist = []
#x_trainstack = 
x_trainlabels = []
#x_train =
#x_test =
#y_train =
#y_test =

In [4]:
#Testing Xarray Version here!
#for i in range(no_classes):
#    data = train.to_xarray()
#    traindatalist.append(data)
#trainstack = xarray.concat(traindatalist, dim='i')

for i in range(no_classes):
    traindatalist.append(train)

trainstack = xr.concat([df.to_xarray() for df in traindatalist], dim="Class")
del train
del traindatalist

In [9]:
trainstack.keys()

KeysView(<xarray.Dataset>
Dimensions:   (Class: 10, index: 42000)
Coordinates:
  * index     (index) int64 0 1 2 3 4 5 ... 41994 41995 41996 41997 41998 41999
Dimensions without coordinates: Class
Data variables:
    label     (Class, index) int64 1 1 1 1 1 1 1 1 1 1 1 ... 0 0 0 0 0 0 0 0 0 0
    pixel0    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel1    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel2    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel3    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel4    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel5    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel6    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel7    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel8    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 .

In [8]:
#Convert the label column to binary depending on what number is represented
for i in range(no_classes):
    #trainlist[i].at[(trainlist[i].label == int(i))] = 1
    #trainstack.loc[(i,slice(None)),'label'] = np.where((trainstack.loc[(i,slice(None)),'label']==i), 1, 0)
    
    #trainstack.loc[dict(Class=5)]['label'] = np.where((trainstack.loc[dict(Class=5)]['label']==i), 1, 0)
    
    #trainstack[dict(Class=i)]['label'].values = np.where((trainstack[dict(Class=i)]['label']==i), 1, 0)
    mask = ((trainstack["Class"] == i) & (trainstack["label"] == i))
    antimask = ((trainstack["Class"] == i) & (trainstack["label"] != i))
    
    trainstack["label"] = trainstack["label"].where(mask, 0, drop=False)
    trainstack["label"] = trainstack["label"].where(antimask, 1, drop=False)

    #trainstack.where(trainstack['Class'] == i and trainstack[:,'label'] != i, 1)
    #trainlist[i].at[(trainlist[i].label != int(i))] = 0
   
    

In [12]:
trainstack['label']

<xarray.DataArray 'label' (Class: 10, index: 42000)>
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)
Coordinates:
  * index    (index) int64 0 1 2 3 4 5 6 ... 41994 41995 41996 41997 41998 41999
Dimensions without coordinates: Class

In [ ]:
#x_trainstack.loc[(i,slice(None))].values.reshape(-1, image_width, image_height, 1)

In [14]:
#Convert data to correct format for training
y_trainstack = trainstack['label']
#x_trainstack = trainstack.drop(['label'], axis = 1)
x_trainstack = trainstack.drop(labels='label')
#del trainstack


In [15]:
x_trainstack

<xarray.Dataset>
Dimensions:   (Class: 10, index: 42000)
Coordinates:
  * index     (index) int64 0 1 2 3 4 5 ... 41994 41995 41996 41997 41998 41999
Dimensions without coordinates: Class
Data variables:
    pixel0    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel1    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel2    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel3    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel4    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel5    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel6    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel7    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel8    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    pixel9    (Class, index) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 

In [16]:
y_trainstack

<xarray.DataArray 'label' (Class: 10, index: 42000)>
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)
Coordinates:
  * index    (index) int64 0 1 2 3 4 5 6 ... 41994 41995 41996 41997 41998 41999
Dimensions without coordinates: Class

In [ ]:
#Reshape x_train data to (10, 42000, 28, 28)

## TODO Fix data reshaping
#for i in range(no_classes):
#    x_trainstack.loc[(i,slice(None))] = x_trainstack.loc[(i,slice(None))].values.reshape(42000, image_width, image_height)
#x_trainstack = x_trainstack.values.reshape((:, :, image_width, image_height))
for i in range(no_classes):
     x_trainstack.loc[(i,slice(None))] = x_trainstack.loc[(i,slice(None))].values.reshape(x_trainstack.loc[(i,slice(None))].shape[0],image_width, image_height, 1)
#    x_trainlist[i] = x_trainlist[i].values.reshape(x_trainlist[i].shape[0], image_width, image_height, 1)
#    x_trainlist[i] = x_trainlist[i].astype('float32')
#
#for i in range(len(x_trainlist)):
#    x_trainlist[i] = x_trainlist[i]/255
#    
#for i in range(len(x_trainlist)):
#    x_train[i], x_test[i], y_train[i], y_test[i] = train_test_split(x_trainlist[i], y_trainlist[i], test_size = validation_split)
x_train, x_test, y_train, y_test = train_test_split(x_trainstack, y_trainstack, test_size = validation_split)

In [ ]:
#Verify the data shape
x_train.shape

In [ ]:
#Verify label shape
y_train.shape

In [ ]:
#Define the models
models = list()
for i in range(no_classes):
    model = Sequential()
    model.add(Conv2D(filters = (base_filters/2), kernel_size = (5,5), padding = 'Same', activation = 'relu', input_shape = (image_width, image_height, 1)))
    model.add(Conv2D(filters = (base_filters/2), kernel_size = (5,5), padding = 'Same', activation = 'relu'))
    model.add(Conv2D(filters = (base_filters/2), kernel_size = (5,5), padding = 'Same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = (base_filters), kernel_size = (3,3), padding = 'Same', activation = 'relu'))
    model.add(Conv2D(filters = (base_filters), kernel_size = (3,3), padding = 'Same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(64, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation = "softmax"))
    models.insert(i, model)
models[0].summary()

In [ ]:
#Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [ ]:
#Compile the models
for i in range(len(models)):
    models[i].compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
#Set LRR
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3, verbose = 1, factor = 0.5, min_lr = 0.00001)

In [ ]:
x_train.loc[(i,slice(None))]

In [ ]:
#train the individual models for each number
histories = list()
for i in range(len(models)):
    histories.insert(i, models[i].fit(x_train.loc[(i,slice(None))], y_train.loc[(i,slice(None))], batch_size = batch_size, epochs = no_epochs,verbose = 1, validation_data = (x_test.loc[(i,slice(None))], y_test.loc[(i,slice(None))]), callbacks = [learning_rate_reduction]))